# Exploration tool for assessment schemes in ICNIRP 2020 and RPS S-1
author: Dr Vitas Anderson (*Two Fields Consulting*)

date: 22/6/2021

The function below is used to calculate the whole body point spatial distribution over height z:

$\Large L_{wbps} = \frac{k_1}{\textrm{cosh}(k_2(z-z_{source}))}$

## Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from collections import namedtuple
from collections.abc import Iterable
pd.options.display.max_rows = 201

# use this for interactive plots
%matplotlib ipympl

# use this for static inline plots
# %matplotlib inline

## Functions

In [10]:
def makelist(obj):
    '''make sure obj is a list'''
    if isinstance(obj, Iterable):
        return list(obj)
    else:
        return [obj]    

def ass_calc(h,nsap,fMHz,zlow,zhigh,dz,k1s,k2s,source_heights):
    '''Function which calculates distributions over a set z height range for:
       + point spatial limit normalised field values for whole body exposure (Lwbps)
       + point spatial limit normalised field values for local exposure (Llocps)
       + spatial average of the whole body Lps points (Lwbsa) over a vertical line
       + point spatial representation of the whole body spatial averages (Lwbpssa)
       
       FUNCTION INPUTS:       
          h = height of the spatial averaging window (m)
       nsap = number of spatial averaging points [3 or 5]
       fMHz = exposure frequency in MHz
       zlow = lower bound of the z points (m)
      zhigh = upper bound of the z points (m)
        k1s = list of k1 parameters that set the peak level of the Lps distribution, 
              e.g. 2 for a single source or [2,1] for two sources
        k2s = list of k2 parameters that set the vertical beamwidth of the Lps distribution
              e.g. 1 for a single source or [1,2] for two sources
        source_heights = list of heights (m) for the source(s), 
              e.g. 7 for a single source, or [10,13] for two sources
         
       FUNCTION OUTPUTS:
         a named tuple, L, containing:
           + title: an informative title for the assessment
           + fMHz: exposure frequency in MHz
           + labels: lables for each calculated distribution
           + z: the z distribution points for the assessment
           + zlow: the lower bound of the z distribution
           + zhigh: the upper bound of the z distribution
           + lass: a named tuple for the calculated assessments (Lwbps, Llocps, Lwbsa, Lwbpssa)
           + nh: number of points in teh vertical averaging window
           + df: a pandas dataframe for z and Lwbps, Llocps, Lwbsa, Lwbpssa
      '''

    ## Generate z distribution
    nz = int((zhigh-zlow) / dz) + 1     # number of z points
    z = np.linspace(zlow, zhigh, nz) 

    ## Make sure that all the source parameters are lists which are of the same length
    k1s = makelist(k1s)
    k2s = makelist(k2s)
    source_heights = makelist(source_heights)   
    assert len(k1s) == len(k2s) == len(source_heights), 'k1s, k2s and source_heights must all have the same number of elements'
    
    ## Generate whole body point spatial distribution (Lwbps)
    Lwbps = np.zeros(nz)                  # initialise Lwbps array with zeros
    for k1, k2, zs in zip(k1s, k2s, source_heights):  # loop through each source
        Lwbps += k1 / np.cosh(k2*(z-zs))  # add artificial distribution of Lwbps for source i

    ## Generate point spatial distribution for local exposure (Llocps)
    if fMHz <= 400:
        m = 5
    elif fMHz < 2000:
        m = 11.47459891 * fMHz**-0.138646884
    else:
        m = 4
        
    Llocps = Lwbps / m

    ## Calculate the wb spatial average (Lwbsa) 
    nh  = int(h/dz)    # number of dz intervals within spatial averaging window
    nh2 = int(nh/2)    # number of dz intervals within half spatial averaging window

    errmsg = f'Change h ({h}), nsap ({nsap}), &/or dz ({dz}) so that spatial averaging points align with z distribution points'
    assert nsap in [3,5], f'nsap ({nsap}) must be 3 or 5'
    
    # spatial averaging indices for 3 point spatial averages
    if nsap == 3:
        ni = int(nh / 2)   # number of dz intervals between spatial averaging points
        assert 2*ni*dz == h, errmsg  
        hindices = [-ni, 0, ni]    
    # spatial averaging indices for 5 point spatial averages
    elif nsap == 5:
        ni = int(nh / 4)   # number of dz intervals between spatial averaging points
        assert 4*ni*dz == h, errmsg  
        hindices= [-2*ni, -ni, 0, ni, 2*ni]
        
    # convert list of indices into a numpy array    
    hindices = np.array(hindices)  

    # initialise Lwbsa array with NaN's (Not a Number)
    Lwbsa = np.repeat(np.nan, nz)
    
    # calculate the mean value for the spatial averaging indices 
    for iz in range(nh2, nz-nh2):
        Lwbsa[iz] = Lwbps[hindices + iz].mean()

    ## Calculate the point spatial distribution of the wb spatial average (Lwbpssa)
    Lwbpssa = np.repeat(np.nan, nz) # initialise Lwbpssa array with NaN's (Not a Number)
    
    # first pass: assign Lwbpssa values for head or feet exposure based on slope of Lwbsa curve
    for iz in range(nh, nz-nh):       
        # Lwbsa values increasing with height (head exposure)
        if Lwbsa[iz+1] > Lwbsa[iz-1]:  
            Lwbpssa[iz] = Lwbsa[iz-nh2]
            
        # Lwbsa values decreasing with height (feet exposure)
        else:
            Lwbpssa[iz] = Lwbsa[iz+nh2]
    
    # second pass: assign max Lwbsa value at the point where exposure Lwbsa peaks,
    # where spatial averaging window exposure changes between head and feet
    ipeaks, _ = find_peaks(Lwbsa)  # get indices of local peaks of Lwbsa values    
    for iz in ipeaks:
        Lwbpssa[iz] = Lwbsa[iz]
        
    # third pass: assign minimum Lwbsa value for the length of the spatial averaging window
    # where Lwbsa is a minimum
    itroughs, _ = find_peaks(1/Lwbsa)  # get indices of local minimums of Lwbsa values
    for iz in itroughs:
        Lwbpssa[(iz-nh2):(iz+nh2)] = Lwbsa[iz]
            
    ## Create pandas dataframe of z and all assessment distributions
    df = pd.DataFrame(dict(z=z,Lwbps=Lwbps,Llocps=Llocps,Lwbsa=Lwbsa,Lwbpssa=Lwbpssa))
    
    ## Create labels for the L assessment distributions
    labels = ['wb point spatial',
              'local point spatial',
              'wb spatial average',
              'wb point spatial spat. avg.']
    
    ## Create a title for the data set
    title = f'{nsap} points over {h}m\n' + '$k_1$' + f'={",".join(map(str,k1s))}, ' + '$k_2$' + f'={",".join(map(str,k2s))}, f={fMHz} MHz'
    
    ## Create a named tuple for all the L assessment distributions
    Lass = namedtuple("Lass", "Lwbps, Llocps, Lwbsa, Lwbpssa")
    lass = Lass(Lwbps, Llocps, Lwbsa, Lwbpssa)
    
    ## Create a named tuple for all the output data
    Ldata = namedtuple("Ldata", "title fMHz labels z zlow zhigh lass nh df")
    ldata = Ldata(title, fMHz, labels, z, zlow, zhigh, lass, nh, df)

    return ldata

def plotL(L):
    '''Plot the L distributions'''
    fig, ax = plt.subplots(figsize=(4,8))
    for lass, label in zip(L.lass, L.labels):
        ax.plot(lass, L.z, alpha=0.6, label=label)
#     ax.lines[1].remove()
#     ax.lines[0].remove()
    ax.set_xlabel('Limit normalised field value')
    ax.set_ylabel('z (m)')
    ax.grid(ls='--')
    ax.legend()
    zmin, zmax = int(L.zlow), int(L.zhigh)
    ax.set_yticks(range(zmin,zmax))
    ax.set_ylim(zmin,zmax)
    ax.set_title(L.title)
    fig.tight_layout()
    
    return ax

## Explore different configurations

In [13]:
plt.close('all')
L1 = ass_calc(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.01,
              k1s=[1,2], k2s=[2,2],
              source_heights=[4, 8])

plotL(L1);
plt.savefig("../plots/2 source 5 pts far peaks.png",dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.close('all')
L1 = ass_calc(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.01,
              k1s=[1,2], k2s=[2,2],
              source_heights=[4, 6])

plotL(L1);
plt.savefig("../plots/2 source 5 pts close peaks.png",dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
plt.gcf().savefig("../plots/2 source 5 pts.png",dpi=300)

In [31]:
plt.close('all')
L1 = ass_calc(h=2, nsap=3, fMHz=400,
              zlow=0, zhigh=12, dz=0.1,
              k1s=[1]*2, k2s=[2]*2,
              source_heights=[4, 8])

plotL(L1);

50 70


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary $k_1$ and $k_2$ 

In [32]:
fig, axes = plt.subplots(1,4,figsize=(9,8))
for j, (k1,k2) in enumerate(zip([1,1,1,2],[0.5,1,2,1])):
    
    # Calculate the assesments
    L = ass_calc(h=2, nsap=5, fMHz=400,
                 zlow=0, zhigh=20, dz=0.1,
                 k1s=k1, k2s=k2,
                 source_heights=10)
    
    # Create the plots
    ax = axes[j]
    ax.plot(L.lass.Lwbps, L.z, alpha=0.7, label=L.labels[0])
    ax.set_xlabel('Limit normalised field value')
    ax.set_ylabel('z (m)')
    ax.grid(ls='--')
    ax.legend(fontsize=8,loc='upper right')
    zmin, zmax = int(L.zlow), int(L.zhigh)
    ax.set_yticks(range(zmin,zmax))
    ax.set_ylim(zmin,zmax)
    ax.set_title(L.title, fontsize=10)
    
fig.tight_layout(w_pad=2)
# fig.savefig('../plots/k1 k2 plots.png', dpi=100)
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary nsap and $k_2$ for a <u>single</u> source

In [34]:
fig, axes = plt.subplots(2,3,figsize=(9,12))
for i, nsap in enumerate([3,5]):
    for j, k2 in enumerate([2, 1, 0.5]):
        
        # Calculate the assessments
        L = ass_calc(h=2, nsap=nsap, fMHz=400,
                     zlow=0, zhigh=20, dz=0.01,
                     k1s=1, k2s=k2,
                     source_heights=10)
        
        # Create the plots 
        ax = axes[i,j]
        for lass, label in zip(L.lass, L.labels):
            ax.plot(lass, L.z, alpha=0.7, label=label)
        ax.set_xlabel('Limit normalised field value')
        ax.set_ylabel('z (m)')
        ax.grid(ls='--')
        ax.legend(fontsize=8,loc='upper right')
        zmin, zmax = int(L.zlow), int(L.zhigh)
        ax.set_yticks(range(zmin,zmax))
        ax.set_ylim(zmin,zmax)
        ax.set_title(L.title,fontsize=10)
        
    fig.tight_layout(h_pad=4,w_pad=2)

# fig.savefig('../plots/nsap k2 plots 1source.png', dpi=100)        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary nsap and $k_2$ for <u>two</u> sources

In [35]:
fig, axes = plt.subplots(2,3,figsize=(9,12))
for i, nsap in enumerate([3,5]):
    for j, k2 in enumerate([2, 1, 0.5]):

        # Calculate the assessments
        L = ass_calc(h=2, nsap=nsap, fMHz=400,
                     zlow=0, zhigh=20, dz=0.01,
                     k1s=[1]*2, k2s=[k2]*2,
                     source_heights=[7,13])
        
        # Create the plots
        ax = axes[i,j]
        for lass, label in zip(L.lass, L.labels):
            ax.plot(lass, L.z, alpha=0.7, label=label)
        ax.set_xlabel('Limit normalised field value')
        ax.set_ylabel('z (m)')
        ax.grid(ls='--')
        ax.legend(fontsize=8,loc='upper right')
        zmin, zmax = int(L.zlow), int(L.zhigh)
        ax.set_yticks(range(zmin,zmax))
        ax.set_ylim(zmin,zmax)
        ax.set_title(L.title)
    fig.tight_layout(h_pad=4,w_pad=2)

# fig.savefig('../plots/nsap k2 plots 2source.png', dpi=100)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

900 1100
900 1100
900 1100
900 1100
900 1100
900 1100


## Scratch
*for testing out code ...*

In [36]:
# find local peaks
ipeaks, _ = find_peaks(L1.lass.Lwbsa)
display(L1.df.iloc[ipeaks,:])

,z,Lwbps,Llocps,Lwbsa,Lwbpssa
40,4.0,1.000671,0.200134,0.512441,0.512441
80,8.0,1.000671,0.200134,0.512441,0.512441


In [38]:
# find local troughs
ipeaks, _ = find_peaks(1/L1.lass.Lwbsa)
display(L1.df.iloc[ipeaks,:])

,z,Lwbps,Llocps,Lwbsa,Lwbpssa
60,6.0,0.073238,0.014648,0.204919,0.204919


#### Create plot of what I *think* the $L_{wbpssa}$ curve should look like

In [39]:
# Create an assessment 
L2 = ass_calc(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.1,
              k1s=[4]*2, k2s=[2]*2,
              source_heights=[4, 8])

# Extract assessment details
Lwbsa = L2.lass.Lwbsa
z = L2.z
nz = len(z)
nh = L2.nh
nh2 = int(nh/2)

# Create curve for upper averaging window bound of the Lwbsa curve 
Lup = np.repeat(np.nan,nz)
Lup[nh2:] = Lwbsa[:-nh2]

# Create curve for lower averaging window bound of the Lwbsa curve 
Llow = np.repeat(np.nan,nz)
Llow[:-nh2] = Lwbsa[nh2:]

# Create curve for what I *think* the Lwbpssa curve should be
Lwbpssa = np.repeat(np.nan,nz)
mask = z >= 8
Lwbpssa[mask] = Llow[mask]
mask = (z < 8) & (z > 7)
Lwbpssa[mask] = Lup[mask]
mask = (z <= 7) & (z >= 5)
Lwbpssa[mask] = Lwbsa[z==6]
mask = (z < 5) & (z >= 4)
Lwbpssa[mask] = Llow[mask]
mask = z < 4
Lwbpssa[mask] = Lup[mask]
mask = z == 8
Lwbpssa[mask] = Lwbsa[mask]
mask = z == 4
Lwbpssa[mask] = Lwbsa[mask]

# Plot curves
c = 'palegreen'
fig, ax = plt.subplots(figsize=(4,8))
ax.plot(Lwbsa, z, color='g', lw=2, label='Lwbsa')
ax.plot(Lup, z, alpha=0.4, color=c,ls='-',label=None)
ax.plot(Llow, z, alpha=0.4, color=c,ls='--',label=None)
ax.plot(Lwbpssa, z, color='r', lw=2, label='Lwbpssa')
ax.vlines(Lwbsa, z-1, z+1, color=c,alpha=0.3,lw=3)
ax.set_xlabel('Limit normalised field value')
ax.set_ylabel('z (m)')
ax.grid(ls='--')
ax.legend()
zmin, zmax = int(L2.zlow), int(L2.zhigh)
ax.set_yticks(range(zmin,zmax))
ax.set_ylim(zmin,zmax)
ax.set_title(L2.title)
fig.tight_layout()




50 70


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Martin's scheme

In [15]:
# Create an assessment 
L3 = ass_calc(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=17, dz=0.01,
              k1s=[3,2], k2s=[2,1],
              source_heights=[4, 10])

# Extract assessment details
Lwbsa = L3.lass.Lwbps
z = L3.z
nz = len(z)
nh = L3.nh
nh2 = int(nh/2)

plotL(L3)
plt.gca().vlines(1,L3.zlow,L3.zhigh,color='r',ls='--');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
plt.savefig('../plots/Martin example.png', dpi=100)        